# Project 3

Required libraries imported below

In [1]:
import matplotlib as plt
import numpy as np
import random

Data Generation

In [2]:
def is_dangerous(color_choices):
    if 3 in color_choices and 1 not in color_choices:
        return True
    return False

def one_hot_encode(color):
    arr = np.zeros(4)    
    arr[color-1] = 1
    return arr
    
def generate_wiring_diagram(part): #part signfies the part of the project the data is being generated for (1 or 2)
    if part != 1 and part != 2:
        print("invalid parameter values entered while calling generate_wiring_diagram(part) function, part can either be 1 or 2")
        return
    
    diagram = np.zeros((20,20,4), dtype=int) # Initializing a 20x20 blank diagram
    color_choices = [1, 2, 3, 4] # 1: Red, 2: Blue, 3: Yellow, 4: Green
    row_choices = list(range(0,20))
    col_choices= list(range(0,20))
    dangerous = False
    wire_to_cut = 0 # 0: No wire to cut, 1: Red, 2: Blue, 3: Yellow, 4: Green

    start_with_row = random.choice([True, False])
    
    if start_with_row: # True: Start with row, False: Start with column
        for i in range(4):
            if i%2 == 0:
                row = random.choice(row_choices) # Choosing a random row
                color = random.choice(color_choices) # Choosing a random color
                diagram[row, :] = one_hot_encode(color)
                row_choices.remove(row)
                color_choices.remove(color)
                if not dangerous:
                    dangerous = is_dangerous(color_choices) # checking if the diagram is dangerous based on the remaining colors
            else:
                col = random.choice(col_choices) # Choosing a random column
                color = random.choice(color_choices) # Choosing a random color
                diagram[:, col] = one_hot_encode(color)
                col_choices.remove(col)
                color_choices.remove(color)
                if i == 1 and not dangerous:
                    dangerous = is_dangerous(color_choices) # checking if the diagram is dangerous based on the remaining colors
                    
            if dangerous and i==3: # If the diagram is dangerous, the third wire placed is the one to cut
                wire_to_cut = color
                
    else:
        for i in range(4):
            if i%2 == 0:
                col = random.choice(col_choices) # Choosing a random column
                color = random.choice(color_choices) # Choosing a random color
                diagram[col, :] = one_hot_encode(color)
                col_choices.remove(col)
                color_choices.remove(color)
                if not dangerous:
                    dangerous = is_dangerous(color_choices) # checking if the diagram is dangerous based on the remaining colors
            else:
                row = random.choice(row_choices) # Choosing a random row
                color = random.choice(color_choices) # Choosing a random color
                diagram[:, row] = one_hot_encode(color)
                row_choices.remove(row)
                color_choices.remove(color)
                if i == 1 and not dangerous:
                    dangerous = is_dangerous(color_choices) # checking if the diagram is dangerous based on the remaining colors
                    
            if dangerous and i==3:
                wire_to_cut = color # If the diagram is dangerous, the third wire placed is the one to cut

    if part == 1: 
        return diagram, dangerous # If part is 1, return the diagram and whether it is dangerous or not
    elif part == 2:
        return diagram, wire_to_cut # If part is 2, return the diagram and the wire to cut
        


Memory Class to store dataset

In [3]:
# Creating a class to create our "dataset" for model training
class Memory:
    # Constructor
    def __init__(self, max_memory):
        self.max_memory = max_memory # maximum amount of samples to remember
        self.samples = [] # the samples

    # Adding a sample to memory
    def add_sample(self, diagram_vector, y):
        self.samples.append([diagram_vector, y])
        if len(self.samples) > self.max_memory: # Removing the earliest sample if we reach maximum memory
            self.samples.pop(0)

    # Sampling the samples we have in memory
    def sample(self, no_samples): #no_samples is the number of samples you need
        if no_samples > len(self.samples):
            return random.sample(self.samples, len(self.samples))
        else:
            return random.sample(self.samples, no_samples)

Data generation for part 1

In [ ]:
diagrams = 5000 #number of max diagrams to be stored in Memory class
memory = Memory(diagrams) #more than diagrams and the oldest diagram stored will be removed
for i in range(diagrams):
    diagram, y = generate_wiring_diagram(1) #type:ignore
    memory.add_sample(diagram, int(y == True))
list1 = memory.sample(500)
print(list1[0][1])
arr1 = (list1[0])[0].flatten()
arr2 = (list1[0])[0].reshape(-1,4)
"""print(arr1)
print(arr2)
for i in arr1:
    print(i, end = "")
print()
for i in arr2:
    print(i, end = "")"""

Data generation for part 2

In [4]:
diagrams = 5000 #number of max diagrams to be stored in Memory class
train_size = 1000 #number of samples to be generated
test_size = 1000 #number of samples to be used for testing
memory = Memory(diagrams) #more than diagrams and the oldest diagram stored will be removed

#generating 5000 diagrams and storing them in memory
i = 0
while i != diagrams:
    diagram, y = generate_wiring_diagram(2) #type:ignore
    if y != 0:
        memory.add_sample(diagram, y)
        i += 1
samples = memory.sample(train_size+test_size)

train_samples = samples[:train_size]
test_samples = samples[train_size:]

diagram_arr = []
y_arr = []
for diagrams, y in train_samples:
    diagram_arr.append(diagrams)
    y_arr.append(y)
    
test_diagram_arr = []
test_y_arr = []    
for test_diagrams, test_y in test_samples:
    test_diagram_arr.append(test_diagrams)
    test_y_arr.append(test_y)

y_arr = np.array(y_arr) #converting the y_arr to a numpy array for easier computation
test_y_arr = np.array(test_y_arr) #converting the test_y_arr to a numpy array for easier computation

In [ ]:
input_features = np.column_stack((np.ones(len(arr2)), arr2))
weights = np.random.randn(len(input_features[0]))    
alpha = 0.05 # Left at 0.05 for now
y = np.empty(len(list1), dtype=int)

for x in range(len(y)):
    y[x] = list1[x][1]

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def log(x):
    return np.log10(x)

def costFunction (features, weights, y): #Loss Function
    loss = 0
    for i in range(len(weights)):
        loss += -1 * y[i] * log(sigmoid(np.dot(weights[i], features[i])) - (1-y[i]) * log(sigmoid(1-np.dot(weights[i], features[i]))))
    loss /= 1/len(weights-1)
    return loss

def gradientDescent(features, weights, alpha, y):
    for i in range(len(weights)):
        gradient = 0
        for j in range(len(weights)):
            func = sigmoid(np.dot(features[j], weights))
            gradient += (func-y[i]) * features[j][i]
        weights[i] = weights[i] - alpha *  gradient

def trainModel(features, weights, alpha):
    epochs = 200
    for x in range(epochs):
        gradientDescent(features, weights, alpha, y)

def testModel(features, weights, alpha, y):
    prediction = sigmoid(np.dot(features, weights))
    print(prediction)
    print(costFunction(features, weights, y))

trainModel(input_features, weights, alpha)

testModel([1, 0, 1, 0, 0], weights, alpha, y)

In [5]:
def process_data(diagram_arr, y_arr):
    flattened_diagrams = np.array([diagram.flatten() for diagram in diagram_arr])
    additional_features = np.array([diagram.sum(axis=(0, 1)) for diagram in diagram_arr])
    X_combined = np.concatenate((flattened_diagrams, additional_features), axis=1)
    X = np.c_[X_combined, np.ones(X_combined.shape[0])]
    Y = np.eye(4)[np.subtract(y_arr,1)]
    
    return X, Y

In [6]:
X_train, Y_train = process_data(diagram_arr, y_arr)
print(X_train.shape)
print(Y_train.shape)

(1000, 1605)
(1000, 4)


Part 2 implementation

In [7]:
#defining the softmax function
def softmax(x): 
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / e_x.sum(axis=1, keepdims=True)

In [8]:
class Model():
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.epsilon = 1e-10
        
    def fit(self, X, Y, learning_rate = 0.001, epochs = 10):
        for epoch in range(epochs):
            
            Z = np.dot(X, self.W) + self.b
            A = softmax(Z)
            
            loss = -np.mean(np.sum(Y * np.log(A + self.epsilon), axis=1)) #calculating the loss
            
            #calculating the gradients
            dW = (1/Y.shape[0]) * np.dot(X.T, (A-Y))
            db = (1/Y.shape[0]) * np.sum(A-Y, axis=0)
            
            #updating the weights and biases
            self.W -= learning_rate * dW
            self.b -= learning_rate * db
            
            if i%100 == 0:
                print("Epoch: ", epoch, "Loss: ", loss)
                
    def predict(self, X): #prediction function
        Z = np.dot(X, self.W) + self.b
        probs = softmax(Z)
        return np.argmax(probs, axis=1) + 1

In [9]:
num_classes = 4
num_features = X_train.shape[1]
W = np.random.rand(num_features, num_classes)
b = np.zeros(num_classes)
model = Model(W, b)

Epoch:  0 Loss:  4.459261183298905
Epoch:  1 Loss:  3.5348059403498517
Epoch:  2 Loss:  3.5274122249899635
Epoch:  3 Loss:  2.92514322995158
Epoch:  4 Loss:  3.080244928030408
Epoch:  5 Loss:  2.6911312635074593
Epoch:  6 Loss:  2.862208365295396
Epoch:  7 Loss:  2.7638104839697046
Epoch:  8 Loss:  2.8966338893791916
Epoch:  9 Loss:  2.9111502938313008
Epoch:  10 Loss:  2.9978259710586586
Epoch:  11 Loss:  2.9518273045835577
Epoch:  12 Loss:  3.0189203335725296
Epoch:  13 Loss:  2.9638830619287404
Epoch:  14 Loss:  3.0203088796508886
Epoch:  15 Loss:  2.956098296195824
Epoch:  16 Loss:  3.0157256768922998
Epoch:  17 Loss:  2.955503247020028
Epoch:  18 Loss:  3.010967592154519
Epoch:  19 Loss:  2.9466713290977014
Epoch:  20 Loss:  3.0058097871960676
Epoch:  21 Loss:  2.944229001582631
Epoch:  22 Loss:  3.0004726729434537
Epoch:  23 Loss:  2.9357676730211937
Epoch:  24 Loss:  2.99536543011279
Epoch:  25 Loss:  2.9323189359248714
Epoch:  26 Loss:  2.989765224798751
Epoch:  27 Loss:  2.924

In [11]:
model.fit(X_train, Y_train, 0.01, 1000)

Epoch:  0 Loss:  0.11490648146861379
Epoch:  1 Loss:  0.11482316096496757
Epoch:  2 Loss:  0.11473995520304676
Epoch:  3 Loss:  0.11465686396704072
Epoch:  4 Loss:  0.11457388704162846
Epoch:  5 Loss:  0.11449102421197878
Epoch:  6 Loss:  0.11440827526375076
Epoch:  7 Loss:  0.114325639983087
Epoch:  8 Loss:  0.11424311815661854
Epoch:  9 Loss:  0.11416070957146078
Epoch:  10 Loss:  0.11407841401521064
Epoch:  11 Loss:  0.11399623127594916
Epoch:  12 Loss:  0.11391416114223606
Epoch:  13 Loss:  0.11383220340311365
Epoch:  14 Loss:  0.11375035784809932
Epoch:  15 Loss:  0.1136686242671901
Epoch:  16 Loss:  0.11358700245085751
Epoch:  17 Loss:  0.11350549219004832
Epoch:  18 Loss:  0.11342409327618234
Epoch:  19 Loss:  0.11334280550115264
Epoch:  20 Loss:  0.11326162865732342
Epoch:  21 Loss:  0.1131805625375266
Epoch:  22 Loss:  0.1130996069350651
Epoch:  23 Loss:  0.11301876164370867
Epoch:  24 Loss:  0.1129380264576933
Epoch:  25 Loss:  0.11285740117171988
Epoch:  26 Loss:  0.11277688

In [12]:
X_test, Y_test = process_data(test_diagram_arr, test_y_arr)
predictions = model.predict(X_test)

correct_predictions = np.sum(predictions == test_y_arr)
print(correct_predictions)
accuracy = (correct_predictions / Y_test.shape[0])*100
print(f"Accuracy: {accuracy}")

for i, predicted_class in enumerate(predictions):
    print(f"Predicted Class - {predicted_class} Actual Class - {test_y_arr[i]}")

967
Accuracy: 96.7
Predicted Class - 3 Actual Class - 3
Predicted Class - 3 Actual Class - 3
Predicted Class - 2 Actual Class - 2
Predicted Class - 4 Actual Class - 4
Predicted Class - 2 Actual Class - 2
Predicted Class - 3 Actual Class - 3
Predicted Class - 3 Actual Class - 3
Predicted Class - 3 Actual Class - 3
Predicted Class - 2 Actual Class - 2
Predicted Class - 3 Actual Class - 3
Predicted Class - 2 Actual Class - 2
Predicted Class - 2 Actual Class - 2
Predicted Class - 3 Actual Class - 3
Predicted Class - 2 Actual Class - 2
Predicted Class - 3 Actual Class - 3
Predicted Class - 4 Actual Class - 4
Predicted Class - 4 Actual Class - 4
Predicted Class - 2 Actual Class - 2
Predicted Class - 4 Actual Class - 4
Predicted Class - 2 Actual Class - 2
Predicted Class - 2 Actual Class - 2
Predicted Class - 3 Actual Class - 3
Predicted Class - 2 Actual Class - 2
Predicted Class - 2 Actual Class - 2
Predicted Class - 3 Actual Class - 3
Predicted Class - 2 Actual Class - 2
Predicted Class - 4

In [ ]:
def min_max_scaling(feature, min_value, max_value):
    return (feature - min_value) / (max_value - min_value)

# Assuming your features are in the list [0, 1, 0, 1, 1, 0, 19, 20, 18]
# Extract the numerical features (assuming the last three are numerical)
numerical_features = [0, 1, 0, 1, 1, 0, 19, 20, 18]


# Apply Min-Max scaling to each feature
scaled_features = [min_max_scaling(feature, min(numerical_features), max(numerical_features)) for feature in numerical_features]

print("Original Features:", numerical_features)
print("Scaled Features:", scaled_features)